### Aliasing
**Learning objectives: using fast fourier transforms, aliasing.**

A noiseless analogue signal can be recovered perfectly from discretely sampled data, provided the sampling rate is greater than double the highest frequency component. This is Nyquist's theorem. If a signal is sampled below this rate, aliasing occurs.

In [81]:
#NAME: Aliasing
#DESCRIPTION: Demonstrates aliasing and the use of fast Fourier transforms.

import numpy as np
from numpy import pi
from numpy.fft import fft, ifft, fftshift

from bokeh.plotting import figure, show, output_notebook
from bokeh.io import push_notebook, gridplot
from ipywidgets import widgets, interact
from IPython.display import display

output_notebook()

sample_no = 256
signal_frequency = 100.0

#time domain function f(t)
t = np.linspace(0.0, 1.0,sample_no)
f = np.cos(2*signal_frequency*pi*t)

Loading BokehJS ...

Time is plotted from 0s to 1s and there are 256 samples. The sampling frequency is therefore 256 Hz, giving a Nyqyuist limit of 128 Hz. Any signal of frequency greater than 128 Hz will be aliased to a frequency below this.

In [82]:
#frequency space domain function F(v) = FT(f(t))
v = np.linspace(-0.5*sample_no, 0.5*sample_no, sample_no)
F = fftshift(abs(fft(f)))

#plot the function F(v)
p1 = figure(x_axis_label = 'Frequency/Hz', y_axis_label = 'FT(cosine)')
q1 = p1.line(v, F)

show(p1)

In [83]:
def change_frequency(signal_frequency):
    f = np.cos(2*signal_frequency*pi*t)
    F = fftshift(abs(fft(f)))
    q1.data_source.data['y'] = F
    push_notebook()

interact(change_frequency, signal_frequency=widgets.IntSlider(min=0,max=256,step=1,value=8));

When the signal frequency exceeds the Nyquist limit it is aliased back to a lower frequency. This can lead to severe distortion of sampled signals. A frequency $f$ in the signal with $f_{sampling} < 2f < 2f_{sampling}$ will 'bounce back' and appear at $2f_{sampling} - f$. 

Consider aliasing of a pair of sinc functions:

In [84]:
hat_width = 100

#time domain signal
top_hat = np.zeros(sample_no)
for i in range(sample_no):
    if abs(i-int(sample_no/2)) < int(0.5*hat_width):
        top_hat[i] = 1.0
    
p2 = figure(height = 300, width = 300, title = 'Top Hat', x_axis_label = 'time/s')
q2 = p2.line(t, top_hat)

In [85]:
#frequency space signal
FT_top_hat = fftshift(fft(top_hat))

p3 = figure(height = 300, width = 300, title = 'Real Part of of FT{top_hat}', x_axis_label = 'frequency/Hz')
q3 = p3.line(v, np.real(FT_top_hat))

In [86]:
#inverse fourier transform back
IFT_FT_top_hat = ifft(FT_top_hat)

p4 = figure(height = 300, width = 300, title = 'Real Part of IFT{FT{top_hat}}', x_axis_label = 'time/s')
q4 = p4.line(t, np.real(IFT_FT_top_hat))

p5 = gridplot([[p2,p3,p4]])
show(p5)

In [87]:
def change_hat_width(hat_width):
    top_hat = np.zeros(sample_no)
    for i in range(sample_no):
        if abs(i-int(sample_no/2)) < 0.5*hat_width:
            top_hat[i] = 1.0
    FT_top_hat = fftshift(fft(top_hat))
    IFT_FT_top_hat = ifft(FT_top_hat)
    #update the data on the graphs
    q2.data_source.data['y'] = top_hat
    q3.data_source.data['y'] = np.real(FT_top_hat)
    q4.data_source.data['y'] = np.real(IFT_FT_top_hat)
    #push this change to the notebook
    push_notebook()

#create an interactive slider to control plot properties
interact(change_hat_width, hat_width=widgets.IntSlider(min=0,max=sample_no,step=1,value=100));

Even at low frequencies, the recovered function (right) appears to have little in common with the original (left). This is because the fourier transform (centre) is truncated by the finite length of the array, 256 in this case. This has the same effect as multiplying the fourier transform by a top hat of width 256 Hz, centred on 0 Hz.

Consequently, `IFT{FT{top_hat}}` is the original `top_hat` convolved with a sinc function of high frequency.
Q: What is this frequency?